In [11]:
from dataset import DictDataset
from model import *

In [12]:
dataset = DictDataset('./data')

from torch.utils.data import DataLoader

dataloader = DataLoader(dataset, batch_size=2, shuffle=True)
sample = dataloader.__iter__().__next__()
bigmodel = BigModel()
pose_final, depth_map = bigmodel(sample)

torch.Size([2, 6])
tensor([[-0.0509,  0.0439,  0.0631, -0.0150,  0.0438,  0.0230],
        [-0.0508,  0.0439,  0.0631, -0.0151,  0.0438,  0.0230]],
       grad_fn=<MeanBackward1>)


In [13]:
class BigModel(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.decoder = DepthDecoder()
        self.posenet = PoseNet(6)
        
    def forward(self, sample):
        pose_final = self.posenet(sample['image_t']['processed_image'], 
                                  sample['image_t1']['processed_image'])
        depth_map = self.decoder(sample['image_t']['feature_embedding'])
        return pose_final, depth_map